In [1]:
import pandas as pd
import numpy as np
import requests
import sqlite3

In [20]:
def insertValue(timestamp, usdtBalance,btcBalance,btcValue,assetValue):
    conn = sqlite3.connect('BackTest.db')
    c = conn.cursor()   
    c.execute('''
    INSERT INTO trend_data (timestamp, usdtBalance, btcBalance,btcValue,assetValue) VALUES (?, ?, ?,?,?)
    ''', (timestamp, usdtBalance, btcBalance, btcValue, assetValue))
    conn.commit()
    conn.close()

def getValue(timestamp):
    conn = sqlite3.connect('BackTest.db')
    c = conn.cursor()
    c.execute('SELECT * FROM trend_data WHERE timestamp = ?', (timestamp,))
    row = c.fetchone()
    conn.close()
    if row:
        return row
    else:
        return (timestamp, 0, 0)  # Example default values; adjust as necessary

def getBalanceAsset(timestamp):
    conn = sqlite3.connect('BackTest.db')
    c = conn.cursor()
    c.execute('SELECT * FROM balanceAsset WHERE timestamp = ?', (timestamp,))
    row = c.fetchone()
    conn.close()
    return row

In [17]:
print(getBalanceAsset(1514840400))

(1514840400, 50000, 3.71863609352816, 50000, 100000)


In [6]:
currentTimestamp = 1514926800
usdtBalance = 100000 #initial balance
btcBalance = 0
assetValue = 0

In [21]:
def currentTrend(timestamp):
    request = getValue(timestamp)
    return request[2]

def btcPrice(timestamp):
    request = getValue(timestamp)
    return requests[1]


def rebalancePortfolio(target_btc_ratio, timestamp, usdtBalance, btcBalance):
    value = getValue(timestamp)
    if value is None or len(value) < 3:
        print(f"No data available for timestamp {timestamp}. Skipping.")
        return  # Skip this timestamp

    current_btc_price = value[1]
    total_value = usdtBalance + (int(btcBalance) * int(current_btc_price))  

    target_btc_value = total_value * target_btc_ratio
    target_usdt_value = total_value * (1 - target_btc_ratio)

    if target_btc_value > (btcBalance * current_btc_price):
        print(f"Buy BTC: {(target_btc_value - (btcBalance * current_btc_price)) / current_btc_price:.2f} units")
    elif target_btc_value < (btcBalance * current_btc_price):
        print(f"Sell BTC: {((btcBalance * current_btc_price) - target_btc_value) / current_btc_price:.2f} units")  # Corrected braces

    if target_usdt_value > usdtBalance:
        print(f"Sell BTC to get USDT: {(target_usdt_value - usdtBalance) / current_btc_price:.2f} units")
    elif target_usdt_value < usdtBalance:
        print(f"Buy USDT: {target_usdt_value - usdtBalance:.2f} units")
    btcBalance = target_btc_value / current_btc_price
    usdtBalance = target_usdt_value
    btcValue = btcBalance *current_btc_price
    finalValue = usdtBalance + btcValue
    conn = sqlite3.connect('BackTest.db')
    c = conn.cursor()
    c.execute('''
        INSERT INTO balanceAsset (timestamp, usdtBalance, btcBalance, btcValue, assetValue)
        VALUES (?, ?, ?, ?, ?)
        ''', (timestamp, usdtBalance, btcBalance, btcValue, finalValue))
    conn.commit()
    conn.close()



# rebalancePortfolio(0.5,1514926800,50000,3.7186)

In [22]:
def main():
    currentTimestamp = 1514926800
    maxTimestamp = 1711656000
    
    for i in range(currentTimestamp, maxTimestamp + 1, 86400):
        xbx_trend = currentTrend(i)  # Fetch the current trend for each day
        
        # Assuming getBalanceAsset correctly fetches the previous day's balance,
        # and rebalancePortfolio has been adjusted to handle 'None' values gracefully
        btc_bal = getBalanceAsset(i-86400)[2] if getBalanceAsset(i-86400) else 0
        usdt_bal = getBalanceAsset(i-86400)[1] if getBalanceAsset(i-86400) else 0
        
        # Now rebalance based on the trend
        if xbx_trend in [0, -0.5, 0.5]:
            rebalancePortfolio(0.5, i, usdt_bal, btc_bal)  # Maintain 50% BTC, 50% USDT
        elif xbx_trend == 1:
            rebalancePortfolio(1.0, i, usdt_bal, btc_bal)  # Go 100% BTC
        elif xbx_trend == -1:
            rebalancePortfolio(0, i, usdt_bal, btc_bal)  # Go 100% USDT, effectively selling all BTC

main()

Sell BTC: 0.53 units
Buy USDT: -2740.00 units


IntegrityError: UNIQUE constraint failed: balanceAsset.timestamp

In [15]:
def main():
    global currentTimestamp
    maxTimestamp = 1711656000
    xbx_trend = currentTrend(currentTimestamp)
    btc_price = btcPrice(currentTimestamp)
    btcValue = btc_price * btcBalance

    xbx_trend = currentTrend(currentTimestamp)

    if (xbx_trend == 0 or xbx_trend == -0.5 or xbx_trend == 0.5):
        rebalancePortfolio(0.5)  # Maintain 50% BTC, 50% USDT
    elif xbx_trend == 1:
        rebalancePortfolio(1.0)  # Go 100% BTC (buy if needed)
    elif xbx_trend == -1:
        rebalancePortfolio(-1.0)  # Sell all USDT and rebalance


main()

TypeError: rebalancePortfolio() missing 3 required positional arguments: 'timestamp', 'usdtBalance', and 'btcBalance'

In [24]:
c.close()

NameError: name 'c' is not defined

In [ ]:
# Update the database
    with create_connection() as conn:
        if conn:
            try:
                with conn.cursor() as cursor:
                    cursor.execute('''
                        INSERT INTO balanceAsset (timestamp, usdtBalance, bitcoinBalance, bitcoinValue, assetValue)
                        VALUES (%s, %s, %s, %s, %s)
                        ''', (timestamp, usdtBalance, btcBalance, btcValue, finalValue))
                    conn.commit()
            except err.MySQLError as e:
                logger.error(f"Failed to insert rebalanced portfolio data: {e}")